In [2]:
import pandas as pd
import numpy as np

from random import sample, seed

from scipy.stats import ttest_ind, chi2_contingency

from Ames_Functions import *

pd.set_option("display.max_columns", None)

In [3]:
df_features_all = load_state_pkl()['df_house_both_after_feature_engineering'][0]

saleprice_2019 = load_state_pkl()["df_RE_latlong2019"][0].SalePrice
saleprice_2021 = load_state_pkl()["df_RE_latlong2021"][0].SalePrice

df_biz = pd.read_csv("data/df_EssentialLatLong_v2.csv")

df_house2019 = load_state_pkl()["df_RE_latlong2019"][0]
df_house2021 = load_state_pkl()["df_RE_latlong2021"][0]
df_house_all = df_house2019.append(df_house2021).set_index("SaleID")

/var/folders/2n/5k95xl754cb_szsgjkmj33k80000gn/T/ipykernel_1113/3715072078.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_house_all = df_house2019.append(df_house2021).set_index("SaleID")


In [4]:
# Manually selecting fields to dummy for AB testing
selected_AB_house_dummies = ['ZngCdPr', 
                            'ClassSc_S', 
                            'Inst1TPr', 
                            'ValType', 
                            'Neighborhood', 
                            'ParType', 
                            'BldgType', 
                            'Foundation', 
                            'RoofMatl', 
                            'Ext1', 
                            'Ext2', 
                            'MasVnrType', 
                            'Heating', 
                            'Central Air',
                            'SaleCond',
                            'Cars', 
                            'MoSold_MM', 
                            'NmbrBRs'
                            ]

selected_AB_house_dummies

['ZngCdPr',
 'ClassSc_S',
 'Inst1TPr',
 'ValType',
 'Neighborhood',
 'ParType',
 'BldgType',
 'Foundation',
 'RoofMatl',
 'Ext1',
 'Ext2',
 'MasVnrType',
 'Heating',
 'Central Air',
 'SaleCond',
 'Cars',
 'MoSold_MM',
 'NmbrBRs']

In [5]:
df_house_all[selected_AB_house_dummies[:-3]]

,ZngCdPr,ClassSc_S,Inst1TPr,ValType,Neighborhood,ParType,BldgType,Foundation,RoofMatl,Ext1,Ext2,MasVnrType,Heating,Central Air,SaleCond
SaleID,,,,,,,,,,,,,,,
92,FS-RL,VACANT LAND - IMPROVED OR UNIMPROVED,WRDJSV,L,HaydnLk,Vac.Lot,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,Normal
93,FS-RL,VACANT LAND - IMPROVED OR UNIMPROVED,WRDJSV,L,HaydnLk,Vac.Lot,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,Normal
96,FS-RL,2-STORY 1946 & NEWER,WRDJSV,F,HaydnLk,Dwg&Lot,1FamDet,C'Block,CompShg,VinylSd,VinylSd,BrkFace,GasFWA,Yes,Normal
98,FS-RL,1-STORY 1946 & NEWER ALL STYLES,WD,F,HaydnLk,Vac.Lot,No Data,No Data,No Data,No Data,No Data,No Data,No Data,No Data,Alloca
100,FS-RL,1-STORY 1946 & NEWER ALL STYLES,WRDJSV,F,HaydnLk,Dwg&Lot,1FamDet,P'Conc,CompShg,VinylSd,VinylSd,Stone,GasFWA,Yes,Alloca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22743,RL,SPLIT OR MULTI-LEVEL,WRDJSV,F,Mitchel,Dwg&Lot,1FamDet,P'Conc,CompShg,HdBoard,Wd Shng,None,GasFWA,No,Normal
22749,RL,1-STORY 1946 & NEWER ALL STYLES,WRDJSV,F,Mitchel,Dwg&Lot,1FamDet,C'Block,CompShg,HdBoard,Plywood,None,GasFWA,Yes,Normal
22756,RL,SPLIT OR MULTI-LEVEL,WD,F,Mitchel,Dwg&Lot,1FamDet,P'Conc,CompShg,HdBoard,Wd Shng,None,GasFWA,Yes,Normal


In [6]:
#Dummying the selected fields and adding them to house data set
df_AB_house_dummies = (pd.get_dummies(df_house_all[selected_AB_house_dummies[:-3]]
                                            .join(df_house_all[["Cars", "MoSold_MM", "NmbrBRs"]]
                                                    .apply(pd.Categorical, axis = 0)
                                                )
                                        )
                        )
              
df_AB_house_dummies

,ZngCdPr_A,ZngCdPr_F-PRD,ZngCdPr_F-VR,ZngCdPr_FS-RL,ZngCdPr_FS-RM,ZngCdPr_GI,ZngCdPr_HOC,ZngCdPr_NC,ZngCdPr_RH,ZngCdPr_RL,ZngCdPr_RM,ZngCdPr_S-SMD,ZngCdPr_UCRM,ClassSc_S_1-1/2 STORY - UNFINISHED ALL AGES,ClassSc_S_1-1/2 STORY CONDO - ALL AGES,ClassSc_S_1-1/2 STORY FINISHED ALL AGES,ClassSc_S_1-1/2 STORY TWNHM - ALL AGES,ClassSc_S_1-LEVEL CONDO 1946 & NEWER,ClassSc_S_1-STORY 1945 & OLDER,ClassSc_S_1-STORY 1946 & NEWER ALL STYLES,ClassSc_S_1-STORY TWNHM - 1946 & NEWER,ClassSc_S_2 FAMILY CONVERSION - ALL STYLES AND AGES,ClassSc_S_2-1/2 STORY ALL AGES,ClassSc_S_2-LEVEL CONDO 1946 & NEWER,ClassSc_S_2-STORY 1945 & OLDER,ClassSc_S_2-STORY 1946 & NEWER,ClassSc_S_2-STORY TWNHM - 1946 & NEWER,ClassSc_S_BED & BREAKFAST,ClassSc_S_DUPLEX - ALL STYLES AND AGES,ClassSc_S_GARAGE UNIT-CONDO,ClassSc_S_MULTILEVEL TWNHM - INCL SPLIT LEV/FOYER,ClassSc_S_SPLIT FOYER,ClassSc_S_SPLIT OR MULTI-LEVEL,ClassSc_S_STORAGE UNIT-CONDO,ClassSc_S_VACANT LAND - IMPROVED OR UNIMPROVED,Inst1TPr_AFFLEST,Inst1TPr_CNDO,Inst1TPr_COD,Inst1TPr_COTAFF,Inst1TPr_CTR,Inst1TPr_D,Inst1TPr_ESMT,Inst1TPr_MSC,Inst1TPr_O2,Inst1TPr_Q,Inst1TPr_SHD,Inst1TPr_SWD,Inst1TPr_WD,Inst1TPr_WRDJSV,ValType_B,ValType_F,ValType_I,ValType_L,ValType_P,Neighborhood_Blmngtn,Neighborhood_Bluestm,Neighborhood_Br'Dale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_DakotaR,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_GrnHill,Neighborhood_HaydnLk,Neighborhood_IDOT&RR,Neighborhood_IOCondo,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_MsCondo,Neighborhood_N Ames,Neighborhood_NRidgHt,Neighborhood_NW Ames,Neighborhood_NoPkVil,Neighborhood_NoRidge,Neighborhood_OldTown,Neighborhood_S&W ISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_Stonebr,Neighborhood_Timber,Neighborhood_Veenker,Neighborhood_WllwCr1,Neighborhood_WllwCr2,ParType_Bld&Lot,ParType_Dwg&Lot,ParType_Vac.Lot,BldgType_1FamDet,BldgType_2FmConv,BldgType_Condo,BldgType_Duplex,BldgType_No Data,BldgType_Twnhs-E,BldgType_Twnhs-I,Foundation_Brk/Til,Foundation_C'Block,Foundation_No Data,Foundation_P'Conc,Foundation_Slab,Foundation_Stone,Foundation_Wood,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_No Data,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Ext1_AsbShng,Ext1_BrkComm,Ext1_BrkFace,Ext1_C'Block,Ext1_CemntBd,Ext1_HdBoard,Ext1_MetalSd,Ext1_No Data,Ext1_Plywood,Ext1_Stucco,Ext1_VinylSd,Ext1_Wd Sdng,Ext1_WdShing,Ext2_AsbShng,Ext2_AsphShn,Ext2_Brk Cmn,Ext2_BrkFace,Ext2_C'Block,Ext2_CmentBd,Ext2_HdBoard,Ext2_ImStucc,Ext2_MetalSd,Ext2_No Data,Ext2_Other,Ext2_Plywood,Ext2_Stone,Ext2_Stucco,Ext2_VinylSd,Ext2_Wd Sdng,Ext2_Wd Shng,MasVnrType_Brk Cmn,MasVnrType_BrkFace,MasVnrType_No Data,MasVnrType_None,MasVnrType_Stone,Heating_GasFWA,Heating_GasHWat,Heating_Geothrm,Heating_Gravity,Heating_HeatPmp,Heating_No Data,Heating_OthElec,Heating_WallHtr,Central Air_No,Central Air_No Data,Central Air_Yes,SaleCond_Abnorml,SaleCond_AdjLand,SaleCond_Alloca,SaleCond_Family,SaleCond_Normal,SaleCond_Partial,Cars_0.0,Cars_0.78,Cars_0.8,Cars_0.85,Cars_1.0,Cars_1.12,Cars_1.16,Cars_2.0,Cars_3.0,Cars_4.0,MoSold_MM_1.0,MoSold_MM_2.0,MoSold_MM_3.0,MoSold_MM_4.0,MoSold_MM_5.0,MoSold_MM_6.0,MoSold_MM_7.0,MoSold_MM_8.0,MoSold_MM_9.0,MoSold_MM_10.0,MoSold_MM_11.0,MoSold_MM_12.0,NmbrBRs_0.0,NmbrBRs_1.0,NmbrBRs_2.0,NmbrBRs_3.0,NmbrBRs_4.0,NmbrBRs_5.0,NmbrBRs_6.0,NmbrBRs_7.0
SaleID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
92,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
93

# Hypothesis Testing (on numeric fields using ttest)

In [7]:
def ttest_2019_2021 (field):
    df_filter = df_features_all.pipe(remove_undeveloped_lots, df_house_all)
    return ttest_ind(df_filter.query('Year == 2019')[field], df_filter.query('Year == 2021')[field])

In [8]:
sig_cols = ['TotalArea_Gar', 'TtlVal_AsrYr', 'YrBuilt', 'Cars', 'Closest_Time', 'Closest_5', 'Convenience_secs', 'Central_historic_feature', 'biz_180_secs', 'biz_480_secs']

sig_fields = (pd.DataFrame([[n, ttest_2019_2021(n)[1]] for n in sig_cols],
               columns = ["field", "p_value"])
                .query("p_value < 0.05").field.values
)
sig_fields

array(['Closest_5', 'Convenience_secs'], dtype=object)

In [9]:
#Showing stats for sig_fields per year
display(df_features_all.query("Year == 2019")[sig_fields].describe(), 
        df_features_all.query("Year == 2021")[sig_fields].describe()
        )

,Closest_5,Convenience_secs
count,901.000000,901.000000
mean,230.535072,370.665927
std,106.177163,122.704920
min,36.920000,145.000000
25%,137.080000,268.300000
50%,213.460000,373.400000
75%,326.120000,462.700000
max,477.300000,640.200000


,Closest_5,Convenience_secs
count,633.000000,633.000000
mean,216.309131,356.659558
std,99.695220,118.336055
min,20.880000,134.600000
25%,136.700000,250.200000
50%,201.840000,360.600000
75%,297.360000,447.500000
max,456.500000,640.200000


In [10]:
#Creating final output table for presentation
(df_features_all
    .pipe(remove_undeveloped_lots, df_house_all)
    .groupby('Year')
    .describe()[sig_fields]
    .T
    .reset_index()
    .query("level_1 == 'mean'")
    .drop("level_1", axis = 1)
    .set_axis(["Feature", "2019", "2021"], axis = 1)
)#.to_csv("Visualizations/Ttest_features_results.csv", index = None)

,Feature,2019,2021
1,Closest_5,226.173187,214.029264
9,Convenience_secs,365.809122,352.444681


# AB Testing (on categorical fields)

In [11]:
#Defining AB Testing function that takes in df_features_all dataset and a specific feature to test 2019 vs 2021 data

def AB_test (df_features_all, feature):
    total = df_features_all.shape[0]
    group2019 = df_features_all.query("Year == 2019").shape[0]
    group2021 = df_features_all.query("Year == 2021").shape[0]
    if df_features_all.groupby("Year")[feature].value_counts().shape[0] < 4:
        return 1,1,1
    group2019_yes = df_features_all.groupby("Year")[feature].value_counts().rename("Counts")[(2019, 1)]
    group2021_yes = df_features_all.groupby("Year")[feature].value_counts().rename("Counts")[(2021, 1)]
    total_yes = group2019_yes + group2021_yes
    total_no = total - total_yes
    group2019_yes_pc = 100 * group2019_yes / group2019
    group2021_yes_pc = 100 * group2021_yes / group2021
    ab_yes_pc = group2019_yes_pc - group2021_yes_pc
    seed(0)
    bag1 = [1]*total_yes + [0]*total_no
    perms = 100
    perm_res = [0]*perms

    for i in range(perms):
        bag2 = sample(bag1, len(bag1))
        group2019_rs = bag2[:group2019]
        group2021_rs = bag2[group2019:]
        group2019_yes_pc_rs = 100 *sum(group2019_rs)/group2019
        group2021_yes_pc_rs = 100 *sum(group2021_rs)/group2021
        perm_res[i] = group2019_yes_pc_rs - group2021_yes_pc_rs

    s_perm_res = pd.Series(perm_res)
    return (sum(s_perm_res.abs() >= abs(ab_yes_pc))/perms, sum(s_perm_res >= ab_yes_pc)/perms, sum(s_perm_res <= ab_yes_pc)/perms)

In [12]:
#Defining features to run AB test on then running AB test function and storing results
selected_AB_features = (df_features_all.filter(regex = "_closest|_GrType|_HsStyle|Has_").columns.tolist() +
                        df_AB_house_dummies.columns.tolist()
                        )

df_features_all_dummies = df_features_all.join(df_AB_house_dummies).join(df_house_all[["YrSold_YYYY", "GLA"]]).pipe(remove_undeveloped_lots)

df_AB_final_results = pd.DataFrame([[result[0], result[1][1], result[1][2]] for result in [[f, AB_test(df_features_all_dummies, f)] for f in selected_AB_features] if result[1][1] <= 0.05 or result[1][2] <= 0.05],
                                         columns = ["Name", "pvalue_2019", "pvalue_2021"])

In [13]:
#Determining which year is favored per feature value
df_AB_Year_Favored = df_AB_final_results.join(df_AB_final_results
                                .apply(lambda x: 2019 if x.pvalue_2019 < x.pvalue_2021 else 2021, axis = 1)
                                .rename("Year_Favored")
                            )      

df_AB_Year_Favored

,Name,pvalue_2019,pvalue_2021,Year_Favored
0,Attachd_GrType,0.00,1.00,2019
1,Detachd_GrType,0.99,0.01,2021
2,None_GrType,0.97,0.04,2021
3,Service_Gym_closest,0.02,0.99,2019
4,Service_Religion_closest,1.00,0.00,2021
5,Service_Restaurant_closest,0.99,0.01,2021
6,Service_Shopping_closest,0.97,0.04,2021
7,ZngCdPr_RH,0.02,1.00,2019
8,ValType_B,1.00,0.03,2021
9,Neighborhood_NoRidge,0.99,0.05,2021


In [14]:
#Formatting final results for output
def print_favored_features (feature):
    year_dict = {2019: [], 2021: []}
    for y in [2019, 2021]: 
        if df_AB_Year_Favored.query("Year_Favored == @y and Name.str.contains(@feature)").shape[0] > 0:
            year_dict[y] = df_AB_Year_Favored[["Name", "Year_Favored"]].query("Name.str.contains(@feature)").pivot(columns = "Year_Favored", values = "Name")[y].dropna().reset_index(drop = True)
    return year_dict

rows_list = []

for f in [ff for ff in selected_AB_house_dummies if df_AB_Year_Favored.query("Name.str.contains(@ff)").shape[0] > 0]:
    results = print_favored_features(f)
    s_f, s_2019, s_2021 = pd.Series([f], name = "Feature"), pd.Series(results[2019], dtype = float, name = 2019), pd.Series(results[2021], dtype = float, name = 2021)
    rows_list.append(pd.concat([s_f, s_2019, s_2021], axis = 1))

In [15]:
#Outputting results to csv file for editting in excel and inputting to presentation
(pd.concat(rows_list)
    .fillna(" ")
    .replace(r"Neighborhood_|ZngCdPr_|ValType_|ParType_|BldgType_|Foundation_|RoofMatl_|Ext1_|Ext2_|Heating_|Central Air_|Cars_|MoSold_MM_|SaleCond_|NmbrBRs_", "", regex = True)
    #.to_csv("Visualizations/Favored_features_all_ABtest.csv", index = False)
)

,Feature,2019,2021
0,ZngCdPr,RH,
0,ValType,,B
0,Neighborhood,SawyerW,NoRidge
1,,,OldTown
0,BldgType,Twnhs-E,
0,Ext1,,WdShing
0,Ext2,,Wd Shng
0,Heating,,Gravity
0,Central Air,Yes,No
0,SaleCond,,Normal


# $Chi^2$ Tests

In [16]:
#This test is specific to the "Number of Houses Sold per Month" plot in Visualizations notebook and slide 8 in presentation
chi2_contingency(df_house_all
    .apply(lambda x: x if x.name != "YrSold_YYYY" else x.astype(int))
    .groupby(["YrSold_YYYY"]).MoSold_MM
    .value_counts()
    .unstack(level = 0)
    .fillna(0)
    .reset_index()
    .query("MoSold_MM < 9.0")
    .filter([2019, 2021])
)[1]

0.04588987198520902

In [17]:
#This test is specific to the "Frequency of Building Types Sold by Year" plot in Visualizations notebook and slide 8 in presentation
chi2_contingency(df_house_all
    .pipe(remove_undeveloped_lots)
    .groupby("YrSold_YYYY").BldgType
    .value_counts(normalize = False)
    .unstack(level = 0)
    .reset_index()
    .filter([2019, 2021])
    .values
)[1]

0.19799938499757733